In [1]:
import os
import openeye.oechem as oechem
import qcportal as ptl
import qcelemental as qcel
import cmiles

## Explore the dataset

In [3]:
# view collections in QCArchive
client = ptl.FractalClient()
client.list_collections()

tagline
collection          name                                                                                   
Dataset             ANI-1                                 22 million off-equilibrium conformations and e...
                    COMP6 ANI-MD                          Benchmark containing MD trajectories from the ...
                    COMP6 DrugBank                        Benchmark containing DrugBank off-equilibrium ...
                    COMP6 GDB10to13                       Benchmark containing off-equilibrium molecules...
                    COMP6 GDB7to9                         Benchmark containing off-equilibrium molecules...
...                                                                                                     ...
TorsionDriveDataset OpenFF Substituted Phenyl Set 1                                                    None
                    Pfizer Discrepancy Torsion Dataset 1                                               None
                    SMIRNOFF Coverage Torsion Set 1                                                    None
                    SiliconTX Torsion Benchmark Set 1                                                  None
                    TorsionDrive Paper                                                                 None

[110 rows x 1 columns]

In [4]:
# pull the collection of interest
ds = client.get_collection('OptimizationDataset', 'OpenFF Full Optimization Benchmark 1')

In [5]:
# view details about group type
ds.list_specifications()

,Description
Name,
default,Standard OpenFF optimization quantum chemistry...


In [6]:
spec = ds.list_specifications().index[0]
ds.list_specifications().iloc[0]['Description']

'Standard OpenFF optimization quantum chemistry specification.'

In [8]:
# see how many molecules in the set (conformers are counted separately)
ds.status(spec)

,default
COMPLETE,26518
ERROR,218


In [9]:
# show the number of optimization steps per entry
ds.counts().head(15)

,default
C[NH2+]C[C@@H](c1ccc(c(c1)O)O)O-0,36.0
C[NH2+]C[C@@H](c1ccc(c(c1)O)O)O-1,19.0
C[NH2+]C[C@@H](c1ccc(c(c1)O)O)O-2,25.0
C[NH2+]C[C@@H](c1ccc(c(c1)O)O)O-3,24.0
C[NH2+]C[C@@H](c1ccc(c(c1)O)O)O-4,28.0
C[NH2+]C[C@@H](c1ccc(c(c1)O)O)O-5,31.0
C[NH2+]C[C@@H](c1ccc(c(c1)O)O)O-6,23.0
C[NH2+]C[C@@H](c1ccc(c(c1)O)O)O-7,24.0
C[NH2+]C[C@@H](c1ccc(c(c1)O)O)O-8,35.0
C[NH2+]C[C@@H](c1ccc(c(c1)O)O)O-9,25.0


## Explore a molecule 

In [10]:
# check out a single molecule and its final geometry
optrec = ds.get_record(name="C[NH2+]C[C@@H](c1ccc(c(c1)O)O)O-0", specification="default")
print(optrec.status)
optrec.get_final_molecule()

RecordStatusEnum.complete


_ColormakerRegistry()

NGLWidget()

In [11]:
optrec.show_history()

In [12]:
optrec.get_final_energy()

-631.6931043923831

## Convert to OEMol, write to file (single mol) 

In [13]:
mol = optrec.get_final_molecule()
type(mol)

qcelemental.models.molecule.Molecule

In [14]:
# convert the qcelemental molecule to an OpenEye molecule
qcjson_mol = mol.dict(encoding='json')
oemol = cmiles.utils.load_molecule(qcjson_mol)
oemol

<openeye.oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0x7f5cebaafe40> >

In [15]:
# convert energy from Hartrees to kcal/mol
ene = optrec.get_final_energy()*qcel.constants.hartree2kcalmol

In [16]:
# add name and energy tag to the mol
oemol.SetTitle("full_1")
oechem.OESetSDData(oemol, "Energy QCArchive", str(ene))

True

In [17]:
# write molecule -- check that title and sd tag exists
ofs = oechem.oemolostream()
ofs.open("test.sdf")
oechem.OEWriteConstMolecule(ofs, oemol)

0

## Convert to OEMol, write to file, test conformer reading

In [21]:
# open an outstream file
outfile = 'test_final.sdf'
ofs = oechem.oemolostream()
if not ofs.open(outfile):
    oechem.OEThrow.Fatal("Unable to open %s for writing" % outfile)

In [22]:
for i, index in enumerate(ds.df.index):
    
    # get the record of each entry
    record = ds.get_record(name=index, specification=spec)
    
    if i%10 == 0: 
        print(i)
    if i == 20:
        break
    print(index)
        
    if record.status == "COMPLETE":
    
        # get optimized molecule of the record
        qc_mol = record.get_final_molecule()

        # convert the qcelemental molecule to an OpenEye molecule
        qcjson_mol = qc_mol.dict(encoding='json')
        oemol = cmiles.utils.load_molecule(qcjson_mol)

        # convert energy from Hartrees to kcal/mol
        ene = record.get_final_energy()*qcel.constants.hartree2kcalmol

        # add name and energy tag to the mol
        oemol.SetTitle(f"full_{i+1}")
        oechem.OESetSDData(oemol, "Energy QCArchive", str(ene))
        
        # write molecule to file
        oechem.OEWriteConstMolecule(ofs, oemol)

ofs.close()

0
C[NH2+]C[C@@H](c1ccc(c(c1)O)O)O-0
C[NH2+]C[C@@H](c1ccc(c(c1)O)O)O-1
C[NH2+]C[C@@H](c1ccc(c(c1)O)O)O-2
C[NH2+]C[C@@H](c1ccc(c(c1)O)O)O-3
C[NH2+]C[C@@H](c1ccc(c(c1)O)O)O-4
C[NH2+]C[C@@H](c1ccc(c(c1)O)O)O-5
C[NH2+]C[C@@H](c1ccc(c(c1)O)O)O-6
C[NH2+]C[C@@H](c1ccc(c(c1)O)O)O-7
C[NH2+]C[C@@H](c1ccc(c(c1)O)O)O-8
C[NH2+]C[C@@H](c1ccc(c(c1)O)O)O-9
10
CO/N=C/1\C[N@](C[C@H]1C[NH3+])c2c(cc3c(=O)c(cn(c3n2)C4CC4)C(=O)[O-])F-0
CO/N=C/1\C[N@](C[C@H]1C[NH3+])c2c(cc3c(=O)c(cn(c3n2)C4CC4)C(=O)[O-])F-1
CO/N=C/1\C[N@](C[C@H]1C[NH3+])c2c(cc3c(=O)c(cn(c3n2)C4CC4)C(=O)[O-])F-2
CO/N=C/1\C[N@](C[C@H]1C[NH3+])c2c(cc3c(=O)c(cn(c3n2)C4CC4)C(=O)[O-])F-3
CO/N=C/1\C[N@](C[C@H]1C[NH3+])c2c(cc3c(=O)c(cn(c3n2)C4CC4)C(=O)[O-])F-4
CO/N=C/1\C[N@](C[C@H]1C[NH3+])c2c(cc3c(=O)c(cn(c3n2)C4CC4)C(=O)[O-])F-5
CO/N=C/1\C[N@](C[C@H]1C[NH3+])c2c(cc3c(=O)c(cn(c3n2)C4CC4)C(=O)[O-])F-6
c1cc(ccc1[C@H]2C[NH2+]CCc3c2cc(c(c3Cl)O)O)O-0
c1cc(ccc1[C@H]2C[NH2+]CCc3c2cc(c(c3Cl)O)O)O-1
c1cc(ccc1[C@H]2C[NH2+]CCc3c2cc(c(c3Cl)O)O)O-2
20


In [230]:
#https://docs.eyesopen.com/toolkits/python/oechemtk/oemol.html
ifs = oechem.oemolistream()
#ifs.SetConfTest(oechem.OEAbsCanonicalConfTest())
ifs.SetConfTest(oechem.OEAbsoluteConfTest(False)) # false means confs may have diff titles
if not ifs.open(outfile):
    raise FileNotFoundError(f"Unable to open {outfile} for reading")
mols = ifs.GetOEMols()

In [231]:
for i, mol in enumerate(mols):
    for j, conf in enumerate(mol.GetConfs()):
        print(i, mol.NumConfs(), conf.GetTitle(), oechem.OEMolToSmiles(conf))

0 10 full_1 C[NH2+]CC(c1ccc(c(c1)O)O)O
0 10 full_2 C[NH2+]CC(c1ccc(c(c1)O)O)O
0 10 full_3 C[NH2+]CC(c1ccc(c(c1)O)O)O
0 10 full_4 C[NH2+]CC(c1ccc(c(c1)O)O)O
0 10 full_5 C[NH2+]CC(c1ccc(c(c1)O)O)O
0 10 full_6 C[NH2+]CC(c1ccc(c(c1)O)O)O
0 10 full_7 C[NH2+]CC(c1ccc(c(c1)O)O)O
0 10 full_8 C[NH2+]CC(c1ccc(c(c1)O)O)O
0 10 full_9 C[NH2+]CC(c1ccc(c(c1)O)O)O
0 10 full_10 C[NH2+]CC(c1ccc(c(c1)O)O)O
1 1 full_14 CON=C1C[NH+](CC1CN)c2c(cc3c(=O)c(cn(c3n2)C4CC4)C(=O)[O-])F
2 1 full_16 CON=C1CN(CC1C[NH3+])c2c(cc3c(=O)c(cn(c3n2)C4CC4)C(=O)[O-])F
3 1 full_17 CON=C1CN(CC1CN)c2c(cc3c(=O)c(cn(c3n2)C4CC4)C(=O)O)F
4 3 full_18 c1cc(ccc1C2C[NH2+]CCc3c2cc(c(c3Cl)O)O)O
4 3 full_19 c1cc(ccc1C2C[NH2+]CCc3c2cc(c(c3Cl)O)O)O
4 3 full_20 c1cc(ccc1C2C[NH2+]CCc3c2cc(c(c3Cl)O)O)O


## Look into inconsistent SMILES strings after conversion

In [23]:
# check out a single molecule and its final geometry
optrec = ds.get_record(name="CO/N=C/1\C[N@](C[C@H]1C[NH3+])c2c(cc3c(=O)c(cn(c3n2)C4CC4)C(=O)[O-])F-3", specification="default")
print(optrec.status)
optrec.get_final_molecule()

RecordStatusEnum.complete


NGLWidget()

In [24]:
# check out a single molecule and its final geometry
optrec = ds.get_record(name="CO/N=C/1\C[N@](C[C@H]1C[NH3+])c2c(cc3c(=O)c(cn(c3n2)C4CC4)C(=O)[O-])F-6", specification="default")
print(optrec.status)
optrec.get_final_molecule()

RecordStatusEnum.complete


NGLWidget()

## Write out the whole set

This took about 5.5 hours

In [ ]:
# open an outstream file
outfile = 'whole.sdf'
ofs = oechem.oemolostream()
if not ofs.open(outfile):
    oechem.OEThrow.Fatal("Unable to open %s for writing" % outfile)

In [ ]:
for i, index in enumerate(ds.df.index):
    
    # get the record of each entry
    record = ds.get_record(name=index, specification=spec)
    
    if i%100 == 0: 
        print(i)
        
    if record.status == "COMPLETE":
    
        # get optimized molecule of the record
        qc_mol = record.get_final_molecule()

        # convert the qcelemental molecule to an OpenEye molecule
        qcjson_mol = qc_mol.dict(encoding='json')
        oemol = cmiles.utils.load_molecule(qcjson_mol)

        # convert energy from Hartrees to kcal/mol
        ene = record.get_final_energy()*qcel.constants.hartree2kcalmol

        # add name and energy tag to the mol
        oemol.SetTitle(f"full_{i+1}")
        oechem.OESetSDData(oemol, "SMILES QCArchive", index)
        oechem.OESetSDData(oemol, "Energy QCArchive", str(ene))
        
        # write molecule to file
        oechem.OEWriteConstMolecule(ofs, oemol)

ofs.close()